# Proyecto Music Stream

Este notebook contiene todas las fases del proyecto:
- Extracción de datos de Spotify y LastFM
- Guardado en CSV
- Diseño y creación de base de datos
- Inserción de datos
- Consultas y análisis SQL

## Instalación de librerías necesarias

Ejecuta la siguiente celda si no tienes instaladas las librerías:

In [49]:
import sqlite3 
import pandas as pd

# Conexión a SQLite
conn = sqlite3.connect('music_stream.db')

# Leer el archivo CSV de artistas de LastFM
artistas_lastfm_df = pd.read_csv('artistas_lastfm.csv')

# Guardar en la base de datos SQLite
artistas_lastfm_df.to_sql('artistas_lastfm', conn, if_exists='replace', index=False)
print("Datos de LastFM insertados en la base de datos SQLite.")

Datos de LastFM insertados en la base de datos SQLite.


## Introduce tus claves de API

Debes obtener tus credenciales de Spotify y LastFM y pegarlas en la siguiente celda.

In [50]:
# Introduce aquí tus claves
API_KEY = '11e26b6792a6279c70bb7b3771993233'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

## Extracción de datos de LastFM

En esta sección se extraen detalles adicionales de artistas y canciones usando la API de LastFM.

In [51]:
import requests
import pandas as pd

# --- CANCIONES ---
artistas = ['Coldplay', 'Adele', 'Shakira', 'Bad Bunny']

def get_lastfm_artist_info(artist_name, api_key):
    url = f'http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artist_name}&api_key={api_key}&format=json'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'artist' in data:
            artist = data['artist']
            return {
                'nombre': artist.get('name'),
                'biografia': artist.get('bio', {}).get('summary'),
                'oyentes': artist.get('stats', {}).get('listeners'),
                'reproducciones': artist.get('stats', {}).get('playcount'),
                'artistas_similares': ', '.join([a['name'] for a in artist.get('similar', {}).get('artist', [])])
            }
    return None

# Extraer la información de los artistas y crear el DataFrame artistas_df
lastfm_data = []
for nombre in artistas:
    info = get_lastfm_artist_info(nombre, API_KEY)
    if info:
        lastfm_data.append(info)
artistas_df = pd.DataFrame(lastfm_data)
artistas_df.to_csv('artistas.csv', index=False)
print('Artistas guardados en artistas.csv')

# --- ÁLBUMES ---
albumes_data = []
for artista in artistas:
    params = {
        'method': 'artist.gettopalbums',
        'artist': artista,
        'api_key': API_KEY,
        'format': 'json',
        'limit': 20
    }
    resp = requests.get(BASE_URL, params=params)
    if resp.status_code == 200:
        albums = resp.json().get('topalbums', {}).get('album', [])
        for album in albums:
            albumes_data.append({
                'id': album.get('mbid', '') or f"{artista}_{album.get('name','')}",  # Si no hay mbid, crea uno propio
                'nombre': album.get('name', ''),
                'artista_id': artista,
                'año': '',  # LastFM no da año aquí
                'reproducciones': album.get('playcount', '')
            })

albumes_df = pd.DataFrame(albumes_data)
albumes_df.to_csv('albumes.csv', index=False)
print('Álbumes guardados en albumes.csv')

# --- CANCIONES ---
canciones_data = []
for artista in artistas:
    params = {
        'method': 'artist.gettoptracks',
        'artist': artista,
        'api_key': API_KEY,
        'format': 'json',
        'limit': 30
    }
    resp = requests.get(BASE_URL, params=params)
    if resp.status_code == 200:
        tracks = resp.json().get('toptracks', {}).get('track', [])
        for track in tracks:
            canciones_data.append({
                'id': track.get('mbid', '') or f"{artista}_{track.get('name','')}",  # Si no hay mbid, crea uno propio
                'nombre': track.get('name', ''),
                'album_id': '',  # No disponible directamente en LastFM
                'artista_id': artista,
                'año': '',  # No disponible directamente en LastFM
                'reproducciones': track.get('playcount', '')
            })

canciones_df = pd.DataFrame(canciones_data)
canciones_df.to_csv('canciones.csv', index=False)
print('Canciones guardadas en canciones.csv')

Artistas guardados en artistas.csv
Álbumes guardados en albumes.csv
Canciones guardadas en canciones.csv


## Guardado de datos en CSV
Guardamos los datos extraídos en archivos CSV para su posterior uso en la base de datos.

In [52]:
import pandas as pd
import os

# Guardar artistas_df 
if 'artistas_df' in globals():
    artistas_df.to_csv('artistas.csv', index=False)
else:
    print('artistas_df no existe')

# Guardar albumes_df 
if 'albumes_df' in globals():
    albumes_df.to_csv('albumes.csv', index=False)
else:
    # Si no existe, crea un DataFrame vacío con columnas ejemplo
    albumes_df = pd.DataFrame(columns=['id','nombre','artista_id','año','reproducciones'])
    albumes_df.to_csv('albumes.csv', index=False)
    print('albumes_df no existe, se ha creado vacío')

# Guardar canciones_df
if 'canciones_df' in globals():
    canciones_df.to_csv('canciones.csv', index=False)
else:
    canciones_df = pd.DataFrame(columns=['id','nombre','album_id','artista_id','año','reproducciones'])
    canciones_df.to_csv('canciones.csv', index=False)
    print('canciones_df no existe, se ha creado vacío')

# Guardar lastfm_df
if 'spotify_df' in globals():
    lastfm_df.to_csv('artistas_spotify.csv', index=False)
else:
    # Si no existe, intenta guardar artistas_df como lastfm_df si tiene las columnas
    if 'artistas_df' in globals() and set(['nombre','biografia','oyentes','reproducciones','artistas_similares']).issubset(artistas_df.columns):
        artistas_df.to_csv('artistas_lastfm.csv', index=False)
        print('lastfm_df no existe, se ha guardado artistas_df como artistas_lastfm.csv')
    else:
        lastfm_df = pd.DataFrame(columns=['nombre','biografia','oyentes','reproducciones','artistas_similares'])
        lastfm_df.to_csv('artistas_lastfm.csv', index=False)
        print('lastfm_df no existe, se ha creado vacío')

print('Archivos CSV de last.fm guardados.')

lastfm_df no existe, se ha guardado artistas_df como artistas_lastfm.csv
Archivos CSV de last.fm guardados.


## Diseño y creación de la base de datos

A continuación, se diseña y crea la base de datos relacional en SQLite para almacenar la información extraída.

In [53]:
import sqlite3
conn = sqlite3.connect('music_stream.db')
c = conn.cursor()
pd.read_sql_query("SELECT * FROM artistas LIMIT 5", conn)

,nombre,biografia,oyentes,reproducciones,artistas_similares
0,Coldplay,Coldplay is a British alternative rock and bri...,8768824,748140383,"Keane, Imagine Dragons, Snow Patrol, OneRepubl..."
1,Adele,"Adele Laurie Blue Adkins MBE (born May 5, 1988...",5391306,303195195,"Sam Smith, Whitney Houston, Beyoncé, Céline Di..."
2,Shakira,Shakira Isabel Mebarak Ripoll is a Colombian s...,4599368,166575113,"Paulina Rubio, Belinda, Thalía, Julieta Venega..."
3,Bad Bunny,"Benito Antonio Martínez Ocasio (born March 10,...",2017111,311922011,"Rauw Alejandro, Jhayco, Mora, Tainy, Myke Towers"


In [54]:
import sqlite3

cnx = sqlite3.connect('music_stream.db')
c = conn.cursor()

# Crear tablas
c.execute("""
    CREATE TABLE IF NOT EXISTS artistas (
        id VARCHAR(255) PRIMARY KEY,
        nombre VARCHAR(255),
        popularidad INT,
        generos TEXT,
        seguidores INT
    )
""")
c.execute("""
    CREATE TABLE IF NOT EXISTS albumes (
        id VARCHAR(255) PRIMARY KEY,
        nombre VARCHAR(255),
        artista_id VARCHAR(255),
        año INT,
        popularidad INT,
        FOREIGN KEY (artista_id) REFERENCES artistas(id)
    )
""")
c.execute("""
    CREATE TABLE IF NOT EXISTS canciones (
        id VARCHAR(255) PRIMARY KEY,
        nombre VARCHAR(255),
        album_id VARCHAR(255),
        artista_id VARCHAR(255),
        año INT,
        popularidad INT,
        FOREIGN KEY (album_id) REFERENCES albumes(id),
        FOREIGN KEY (artista_id) REFERENCES artistas(id)
    )
""")
c.execute("""
    CREATE TABLE IF NOT EXISTS artistas_lastfm (
        nombre VARCHAR(255) PRIMARY KEY,
        biografia TEXT,
        oyentes INT,
        popularidad INT,
        artistas_similares TEXT
    )
""")
cnx.commit()
print("Tablas creadas en MySQL.")

Tablas creadas en MySQL.


In [55]:
# Insertar datos en la base de datos
import pandas as pd
import os
import numpy as np

# Leer los CSV
df_albumes = pd.read_csv('albumes.csv')
df_canciones = pd.read_csv('canciones.csv')

# Reemplaza los NaN y 'nan' por None
df_albumes = df_albumes.replace({np.nan: None, 'nan': None})
df_canciones = df_canciones.replace({np.nan: None, 'nan': None})

if 'reproducciones' in df_canciones.columns:
    df_canciones.rename(columns={'reproducciones': 'popularidad'}, inplace=True)
print(df_canciones.columns)  # Para comprobar que 'popularidad' está presente

# Insertar ALBUMES
for _, row in df_albumes.iterrows():
    c.execute("""
        INSERT OR IGNORE INTO albumes (id, nombre, artista_id, año, reproducciones)
        VALUES (?, ?, ?, ?, ?)
    """, (row['id'], row['nombre'], row['artista_id'], row['año'], row['reproducciones']))
conn.commit()

def safe_read_csv(path, columns):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        try:
            return pd.read_csv(path)
        except Exception as e:
            print(f'Error leyendo {path}:', e)
            return pd.DataFrame(columns=columns)
    else:
        print(f'{path} está vacío o no existe, se crea DataFrame vacío.')
        return pd.DataFrame(columns=columns)

# Insertar CANCIONES
try:
    c.execute("ALTER TABLE canciones ADD COLUMN popularidad INTEGER;")
except Exception as e:
    print("La columna 'popularidad' ya existe o hubo otro error:", e)
for _, row in df_canciones.iterrows():
    c.execute("""
        INSERT OR IGNORE INTO canciones (id, nombre, album_id, artista_id, año, popularidad)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (row['id'], row['nombre'], row['album_id'], row['artista_id'], row['año'], row['popularidad']))
conn.commit()

print("¡Datos de álbumes y canciones insertados en MySQL!")

import pandas as pd

# Leer los archivos CSV
artistas_lastfm_df = pd.read_csv('artistas_lastfm.csv')
albumes_df = pd.read_csv('albumes.csv')
canciones_df = pd.read_csv('canciones.csv')

# Añadir columna 'id' si no existe en álbumes y canciones
if 'id' not in albumes_df.columns:
    albumes_df['id'] = range(1, len(albumes_df) + 1)
if 'id' not in canciones_df.columns:
    canciones_df['id'] = range(1, len(canciones_df) + 1)

# Guardar en la base de datos
artistas_lastfm_df.to_sql('artistas_lastfm', conn, if_exists='replace', index=False)
albumes_df.to_sql('albumes', conn, if_exists='replace', index=False)
canciones_df.to_sql('canciones', conn, if_exists='replace', index=False)
print("Datos de LastFM insertados en la base de datos.")

Index(['id', 'nombre', 'album_id', 'artista_id', 'año', 'popularidad'], dtype='object')
¡Datos de álbumes y canciones insertados en MySQL!
Datos de LastFM insertados en la base de datos.


In [56]:
# Insertar datos en la base de datos
import pandas as pd
import os
import numpy as np

# Leer los CSV
df_albumes = pd.read_csv('albumes.csv')
df_canciones = pd.read_csv('canciones.csv')

# Reemplaza los NaN y 'nan' por None
df_albumes = df_albumes.replace({np.nan: None, 'nan': None})
df_canciones = df_canciones.replace({np.nan: None, 'nan': None})

# Insertar ÁLBUMES
# Añadir columna géneros si no existe
try:
    c.execute("ALTER TABLE albumes ADD COLUMN generos TEXT;")
except:
    pass  # Si ya existe, ignora el error
# Insertar ÁLBUMES con géneros
df_albumes = pd.read_csv('albumes.csv')
df_albumes.rename(columns={'reproducciones': 'popularidad'}, inplace=True)
try:
    c.execute("ALTER TABLE albumes ADD COLUMN popularidad INTEGER;")
except Exception as e:
    print("La columna 'popularidad' ya existe o hubo otro error:", e)
for _, row in df_albumes.iterrows():
    c.execute("""
        INSERT OR IGNORE INTO albumes (id, nombre, artista_id, año, popularidad)
        VALUES (?, ?, ?, ?, ?)
    """, (row['id'], row['nombre'], row['artista_id'], row['año'], row['popularidad']))
conn.commit()

def safe_read_csv(path, columns):
    if os.path.exists(path) and os.path.getsize(path) > 0:
        try:
            return pd.read_csv(path)
        except Exception as e:
            print(f'Error leyendo {path}:', e)
            return pd.DataFrame(columns=columns)
    else:
        print(f'{path} está vacío o no existe, se crea DataFrame vacío.')
        return pd.DataFrame(columns=columns)

# Insertar CANCIONES
df_canciones = pd.read_csv('canciones.csv')
df_canciones.rename(columns={'reproducciones': 'popularidad'}, inplace=True)
try:
    c.execute("ALTER TABLE canciones ADD COLUMN popularidad INTEGER;")
except Exception as e:
    print("La columna 'popularidad' ya existe o hubo otro error:", e)
for _, row in df_canciones.iterrows():
    c.execute("""
        INSERT OR IGNORE INTO canciones (id, nombre, album_id, artista_id, año, popularidad)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (row['id'], row['nombre'], row['album_id'], row['artista_id'], row['año'], row['popularidad']))
conn.commit()

print("¡Datos de álbumes y canciones insertados en MySQL!")

df_albumes = safe_read_csv('albumes.csv', ['id','nombre','artista_id','año','popularidad'])
df_canciones = safe_read_csv('canciones.csv', ['id','nombre','album_id','artista_id','año','popularidad'])
df_lastfm = safe_read_csv('artistas_lastfm.csv', ['nombre','biografia','oyentes','reproducciones','artistas_similares'])

df_albumes.to_sql('albumes', conn, if_exists='replace', index=False)
df_canciones.to_sql('canciones', conn, if_exists='replace', index=False)
df_lastfm.to_sql('artistas_lastfm', conn, if_exists='replace', index=False)
print('Datos insertados en la base de datos.')

¡Datos de álbumes y canciones insertados en MySQL!
Datos insertados en la base de datos.


## Consultas SQL y análisis

A continuación, se realizan consultas SQL para responder a las preguntas del proyecto.

In [57]:
import mysql.connector

# Conexión a MySQL
cnx = mysql.connector.connect(
    user='root',
    password='trabajo5',
    host='127.0.0.1',
    database='music_stream'
)
c = cnx.cursor()
print("Conexión a MySQL realizada correctamente.")

Conexión a MySQL realizada correctamente.


In [58]:
import pandas as pd
import numpy as np

# Leer los CSV
df_artistas = pd.read_csv('artistas.csv')
df_albumes = pd.read_csv('albumes.csv')
df_canciones = pd.read_csv('canciones.csv')

# Reemplaza los NaN y 'nan' por None
df_artistas = df_artistas.replace({np.nan: None, 'nan': None})
df_albumes = df_albumes.replace({np.nan: None, 'nan': None})
df_canciones = df_canciones.replace({np.nan: None, 'nan': None})

# Insertar ARTISTAS en la tabla correcta (artistas_lastfm)
for _, row in df_artistas.iterrows():
    c.execute("""
        INSERT IGNORE INTO artistas_lastfm (nombre, biografia, oyentes, reproducciones, artistas_similares)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['nombre'], row['biografia'], row['oyentes'], row['reproducciones'], row['artistas_similares']))
cnx.commit()

# Insertar ALBUMES
df_albumes.rename(columns={'reproducciones': 'popularidad'}, inplace=True)
for _, row in df_albumes.iterrows():
    c.execute("""
        INSERT IGNORE INTO albumes (id, nombre, artista_id, año, popularidad)
        VALUES (%s, %s, %s, %s, %s)
    """, (row['id'], row['nombre'], row['artista_id'], row['año'], row['popularidad']))
cnx.commit()

# Insertar CANCIONES
df_canciones.rename(columns={'reproducciones': 'popularidad'}, inplace=True)
for _, row in df_canciones.iterrows():
    c.execute("""
        INSERT IGNORE INTO canciones (id, nombre, album_id, artista_id, año, popularidad)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (row['id'], row['nombre'], row['album_id'], row['artista_id'], row['año'], row['popularidad']))
cnx.commit()

print("¡Datos de artistas, álbumes y canciones insertados en MySQL!")

¡Datos de artistas, álbumes y canciones insertados en MySQL!


In [59]:
# 1. Artista con más álbumes
consulta1 = pd.read_sql_query("""
    SELECT artistas.nombre AS artista, COUNT(albumes.id) AS num_albumes
    FROM albumes
    JOIN artistas ON albumes.artista_id = artistas.id
    GROUP BY artistas.nombre
    ORDER BY num_albumes DESC
    LIMIT 1
""", cnx)
display(consulta1)

# 2. ¿Qué artista tiene más reproducciones? 
consulta = pd.read_sql_query("""
    SELECT nombre, reproducciones
    FROM artistas
    WHERE reproducciones IS NOT NULL
    ORDER BY reproducciones DESC
    LIMIT 1
""", cnx)
display(consulta)

#3. Canción mejor valorada
consulta4 = pd.read_sql_query("""
    SELECT nombre, popularidad
    FROM canciones
    WHERE popularidad IS NOT NULL AND popularidad != ''
    ORDER BY CAST(popularidad AS UNSIGNED) DESC
    LIMIT 1
""", cnx)
display(consulta4)

#4. Artista más popular
consulta5 = pd.read_sql_query("""
    SELECT nombre, popularidad
    FROM artistas
    WHERE popularidad IS NOT NULL AND popularidad != ''
    ORDER BY CAST(popularidad AS UNSIGNED) DESC
    LIMIT 1
""", cnx)
display(consulta5)

#5. Top 5 canciones más escuchadas
top_canciones = pd.read_sql_query("""
    SELECT nombre, artista_id, popularidad
    FROM canciones
    WHERE popularidad IS NOT NULL AND popularidad != ''
    ORDER BY CAST(popularidad AS UNSIGNED) DESC
    LIMIT 5
""", cnx)
display(top_canciones)

C:\Users\Rocio\AppData\Local\Temp\ipykernel_12192\52596609.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  consulta1 = pd.read_sql_query("""


,artista,num_albumes
0,Coldplay,10


C:\Users\Rocio\AppData\Local\Temp\ipykernel_12192\52596609.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  consulta = pd.read_sql_query("""


,nombre,reproducciones


C:\Users\Rocio\AppData\Local\Temp\ipykernel_12192\52596609.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  consulta4 = pd.read_sql_query("""


,nombre,popularidad
0,Viva la Vida,38743609


C:\Users\Rocio\AppData\Local\Temp\ipykernel_12192\52596609.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  consulta5 = pd.read_sql_query("""


,nombre,popularidad
0,Bad Bunny,99


C:\Users\Rocio\AppData\Local\Temp\ipykernel_12192\52596609.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_canciones = pd.read_sql_query("""


,nombre,artista_id,popularidad
0,Viva la Vida,Coldplay,38743609
1,Viva la Vida,Coldplay,38743609
2,Viva la Vida,Coldplay,38733041
3,Viva la Vida,Coldplay,38733041
4,Viva la Vida,Coldplay,38733041


In [60]:
c.execute("SELECT COUNT(*) FROM artistas")
print("Artistas:", c.fetchone()[0])
c.execute("SELECT COUNT(*) FROM albumes")
print("Albumes:", c.fetchone()[0])
c.execute("SELECT COUNT(*) FROM canciones")
print("Canciones:", c.fetchone()[0])

Artistas: 4
Albumes: 512
Canciones: 1216


In [61]:
""" 
PONER EN SQL:

CREATE DATABASE music_stream;
CREATE TABLE IF NOT EXISTS artistas (
    id VARCHAR(255) PRIMARY KEY,
    nombre VARCHAR(255),
    popularidad INT,
    generos TEXT,
    seguidores INT
);

CREATE TABLE IF NOT EXISTS albumes (
    id VARCHAR(255) PRIMARY KEY,
    nombre VARCHAR(255),
    artista_id VARCHAR(255),
    año INT,
    popularidad INT,
    FOREIGN KEY (artista_id) REFERENCES artistas(id)
);

CREATE TABLE IF NOT EXISTS canciones (
    id VARCHAR(255) PRIMARY KEY,
    nombre VARCHAR(255),
    album_id VARCHAR(255),
    artista_id VARCHAR(255),
    año INT,
    popularidad INT,
    FOREIGN KEY (album_id) REFERENCES albumes(id),
    FOREIGN KEY (artista_id) REFERENCES artistas(id)
);

CREATE TABLE IF NOT EXISTS artistas_lastfm (
    nombre VARCHAR(255) PRIMARY KEY,
    biografia TEXT,
    oyentes INT,
    popularidad INT,
    artistas_similares TEXT
);

Enlace de la presentación: https://app.genially.com/editor/68b1c8a1019cad2f86825af6"""

' \nPONER EN SQL:\n\nCREATE DATABASE music_stream;\nCREATE TABLE IF NOT EXISTS artistas (\n    id VARCHAR(255) PRIMARY KEY,\n    nombre VARCHAR(255),\n    popularidad INT,\n    generos TEXT,\n    seguidores INT\n);\n\nCREATE TABLE IF NOT EXISTS albumes (\n    id VARCHAR(255) PRIMARY KEY,\n    nombre VARCHAR(255),\n    artista_id VARCHAR(255),\n    año INT,\n    popularidad INT,\n    FOREIGN KEY (artista_id) REFERENCES artistas(id)\n);\n\nCREATE TABLE IF NOT EXISTS canciones (\n    id VARCHAR(255) PRIMARY KEY,\n    nombre VARCHAR(255),\n    album_id VARCHAR(255),\n    artista_id VARCHAR(255),\n    año INT,\n    popularidad INT,\n    FOREIGN KEY (album_id) REFERENCES albumes(id),\n    FOREIGN KEY (artista_id) REFERENCES artistas(id)\n);\n\nCREATE TABLE IF NOT EXISTS artistas_lastfm (\n    nombre VARCHAR(255) PRIMARY KEY,\n    biografia TEXT,\n    oyentes INT,\n    popularidad INT,\n    artistas_similares TEXT\n);\n\nEnlace de la presentación: https://app.genially.com/editor/68b1c8a1019ca

# Proyecto Music Stream
# Objetivo

1. Crear una bases de datos para almacenar información de canciones y artistas. 
2. Extraer información de la base de datos para responder preguntas sobre la música almacenada.

# Fases del proyecto 
**Fase 1: Extraer Datos**
1. Extraer información de Api Spotify.
2. Extraer información de API Last FM.
3. Guardar esa información en CSV.

**Fase 2: Bases de datos**
1. Diseñar la Base de Datos.
2. Crear la bases de datos.
3. Insertar los valores en la bases de datos.
   
**Fase 3: Extraer información**
1. Extraer información de la base de datos para responder preguntas sobre la música almacenada.

## Seguimos... Fase 1.2 Extraer información de Api Last FM

- Hay que ir a la página de Last FM y registrarse para obtener una API KEY. https://www.last.fm/api 
- La dirección URL de la API de Last FM es: http://ws.audioscrobbler.com/2.0/ Así podemos obtener información por artista como: biografía, oyentes, número de reproducciones, artistas similares...

In [62]:
# Importar librerías para manipulación y análisis de datos
!pip install requests
!pip install numpy 
import requests
import pandas as pd

# Importar librería para la conexión con MySQL
!pip install mysql-connector 
from mysql.connector import errorcode

In [63]:
# Este script consulta los artistas en base a una lista que yo le paso (Biography, Listeners, Playcount, Similar Artists...)
# API Key obtenida de last.fm
API_KEY = '11e26b6792a6279c70bb7b3771993233'  # Reemplaza con tu API Key válida de Last.fm
BASE_URL = 'http://ws.audioscrobbler.com/2.0/' # esta base siempre es la misma

### Cómo usar el método de search de Lastfm

In [64]:
import requests

API_KEY = '11e26b6792a6279c70bb7b3771993233'
BASE_URL = 'http://ws.audioscrobbler.com/2.0/'

def search_artist_lastfm(artist_name, api_key):
    params = {
        'method': 'artist.search',
        'artist': artist_name,
        'api_key': api_key,
        'format': 'json',
        'limit': 5  # Puedes ajustar el límite
    }
    response = requests.get(BASE_URL, params=params)
    return response.json()

def search_track_lastfm(track_name, api_key):
    params = {
        'method': 'track.search',
        'track': track_name,
        'api_key': api_key,
        'format': 'json',
        'limit': 10
    }
    response = requests.get(BASE_URL, params=params)
    return response.json()

In [65]:
# Buscar artistas relacionados con "rock"
datos_artistas = search_artist_lastfm('rock', API_KEY)
artistas = datos_artistas['results']['artistmatches']['artist']
for artista in artistas:
    print(artista['name'], '-', artista.get('listeners', ''))
#Aquí vemos que me trae como respuesta y vemos en el resultado que viene dentro de tracks
datos_artistas.keys()

Rock - 67741
Jay Rock - 1328482
PnB Rock - 582781
Kid Rock - 1153693
Aesop Rock - 804289


dict_keys(['results'])

In [66]:
# Buscar canciones relacionadas con "rock"
datos_tracks = search_track_lastfm('rock', API_KEY)
canciones = datos_tracks['results']['trackmatches']['track']
for cancion in canciones:
    print(cancion['name'], '-', cancion['artist'])

Rock - Stepz
Rock - Stray Kids
Rock - Seventeen
Rock - sixzino
Rock - Matt Swi$her
Rock - Cloud Nothings
Rock - Smokingskul
Rock - Plies
Rock - 9lokknine
Rock - Pleymo


In [67]:
datos_artistas

{'results': {'opensearch:Query': {'#text': '',
   'role': 'request',
   'searchTerms': 'rock',
   'startPage': '1'},
  'opensearch:totalResults': '107589',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '5',
  'artistmatches': {'artist': [{'name': 'Rock',
     'listeners': '67741',
     'mbid': 'c4a5630e-9b09-4d9b-9725-ab1fdc096d08',
     'url': 'https://www.last.fm/music/Rock',
     'streamable': '0',
     'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
       'size': 'small'},
      {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
       'size': 'medium'},
      {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
       'size': 'large'},
      {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
       'size': 'extralarge'}]},
    {'name': 'Jay Rock',
     'listeners': '1328482',
     'mbid':

In [68]:
# Acceder al primer track (canción) encontrado
primer_track = datos_tracks['results']['trackmatches']['track'][0]
print(primer_track)

{'name': 'Rock', 'artist': 'Stepz', 'url': 'https://www.last.fm/music/Stepz/_/Rock', 'streamable': '0', 'listeners': '111895', 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'small'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'medium'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'large'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'extralarge'}], 'mbid': ''}


In [69]:
# Acceder al segundo track (canción) encontrado
primer_track = datos_tracks['results']['trackmatches']['track'][2]
print(primer_track)

{'name': 'Rock', 'artist': 'Seventeen', 'url': 'https://www.last.fm/music/Seventeen/_/Rock', 'streamable': '0', 'listeners': '51144', 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'small'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'medium'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'large'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'extralarge'}], 'mbid': '17af9b5a-60d1-47a8-bdc8-4ec74a812bea'}


### Ahora vamos a hacerlo para nuestro problema... ahora sí!  🚀

In [70]:
# Lista para almacenar los datos que luego guardaremos en un DataFrame
artist_data = []
params_info = {
        'method': 'artist.getinfo',
        'artist': "Madonna",
        'api_key': API_KEY,
        'format': 'json'
    }
response_info = requests.get(BASE_URL, params=params_info)
data_info = response_info.json()
data_info

{'artist': {'name': 'Madonna',
  'mbid': '79239441-bfd5-4981-a70c-55c3f15c1287',
  'url': 'https://www.last.fm/music/Madonna',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '0',
  'stats': {'listeners': '5323677', 'playcount': '338504766'},
  'similar': {'artist': [{'name': 'Kylie Minogu

In [71]:
## Vamos a crear un objeto artista 
artista = {
    'Artist': "Madonna",
    'Biography': data_info['artist'].get('bio', {}).get('summary'),
    'Listeners': data_info['artist'].get('stats', {}).get('listeners', "Sin datos"),
    'Playcount': data_info['artist'].get('stats', {}).get('playcount', "Sin datos")
}
artista

{'Artist': 'Madonna',
 'Biography': 'Madonna Louise Veronica Ciccone (born August 16, 1958 in Bay City, Michigan) is an American singer, songwriter and actress. She is considered one of the most influential figures in popular culture and has often been referred to as the "Queen of Pop". Madonna is noted for her continual reinvention and versatility in music production, songwriting and visual presentation. She is also known for pushing the boundaries of artistic expression in mainstream music, while maintaining control over every aspect of her career. <a href="https://www.last.fm/music/Madonna">Read more on Last.fm</a>',
 'Listeners': '5323677',
 'Playcount': '338504766'}

In [72]:
## Para obtener los artistas similares de un artisa
params_similar = {
        'method': 'artist.getsimilar',
        'artist': "Madonna",
        'api_key': API_KEY,
        'format': 'json'
    }
response_similar = requests.get(BASE_URL, params=params_similar)
data_similar = response_similar.json()
data_similar['similarartists']['artist']

[{'name': 'Kylie Minogue',
  'mbid': '2fddb92d-24b2-46a5-bf28-3aed46f4684c',
  'match': '1',
  'url': 'https://www.last.fm/music/Kylie+Minogue',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0'},
 {'name': 'Janet Jackson',
  'mbid': '6be2828f-6c0d-4059-99d4-fa18acf1a296',
  'match': '0.635011',
  'url':

In [73]:
# Hago un listado con los valores similares
similar_artists=[]
for artist in data_similar['similarartists']['artist']:
    similar_artists.append(artist['name'])
similar_artists

['Kylie Minogue',
 'Janet Jackson',
 'Britney Spears',
 'Dannii Minogue',
 'Paula Abdul',
 'Cher',
 'Christina Aguilera',
 'Sophie Ellis-Bextor',
 'Lady Gaga',
 'Jennifer Lopez',
 'Mariah Carey',
 'Robyn',
 'Spice Girls',
 'Donna Summer',
 'Cyndi Lauper',
 'Gwen Stefani',
 'Whitney Houston',
 'Melanie C',
 'Geri Halliwell',
 'Tina Turner',
 'Céline Dion',
 'Rihanna',
 'Taylor Dayne',
 'Shakira',
 'Katy Perry',
 'Róisín Murphy',
 'Miley Cyrus',
 'Grace Jones',
 'Ke$ha',
 'Girls Aloud',
 'Nelly Furtado',
 'Jessie Ware',
 'Kate Bush',
 'Michael Jackson',
 'Cathy Dennis',
 'Fergie',
 'Alanis Morissette',
 'George Michael',
 'Goldfrapp',
 'Barbra Streisand',
 'Mylène Farmer',
 'Sugababes',
 'Shania Twain',
 'Addison Rae',
 'Heidi Montag',
 'Paris Hilton',
 'Annie Lennox',
 'Bananarama',
 'Emma Bunton',
 'Tove Lo',
 'Lindsay Lohan',
 "Destiny's Child",
 'Diana Ross',
 'The Pussycat Dolls',
 'Kelly Osbourne',
 'Rachel Stevens',
 'Beyoncé',
 'Jessica Simpson',
 'Crystal Waters',
 'Deee-Lite',


In [74]:
artista['similar_artists'] = similar_artists
artista

{'Artist': 'Madonna',
 'Biography': 'Madonna Louise Veronica Ciccone (born August 16, 1958 in Bay City, Michigan) is an American singer, songwriter and actress. She is considered one of the most influential figures in popular culture and has often been referred to as the "Queen of Pop". Madonna is noted for her continual reinvention and versatility in music production, songwriting and visual presentation. She is also known for pushing the boundaries of artistic expression in mainstream music, while maintaining control over every aspect of her career. <a href="https://www.last.fm/music/Madonna">Read more on Last.fm</a>',
 'Listeners': '5323677',
 'Playcount': '338504766',
 'similar_artists': ['Kylie Minogue',
  'Janet Jackson',
  'Britney Spears',
  'Dannii Minogue',
  'Paula Abdul',
  'Cher',
  'Christina Aguilera',
  'Sophie Ellis-Bextor',
  'Lady Gaga',
  'Jennifer Lopez',
  'Mariah Carey',
  'Robyn',
  'Spice Girls',
  'Donna Summer',
  'Cyndi Lauper',
  'Gwen Stefani',
  'Whitney H

## Guardar toda la info en un csv

In [75]:
import pandas as pd

# Convertimos la lista de diccionarios de LastFM a un DataFrame
df_lastfm = pd.DataFrame(lastfm_data)

# Guardamos el DataFrame en un archivo CSV
df_lastfm.to_csv('lastfm_data_final.csv', index=False, encoding='utf-8')

In [76]:
# Importar librerías para manipulación y análisis de datos
!pip install requests
!pip install numpy 
import requests
import pandas as pd

In [77]:
# Leer los archivos CSV de LastFM y cargar los datos en la base de datos
artistas_lastfm_df = pd.read_csv('artistas_lastfm.csv')
albumes_df = pd.read_csv('albumes.csv')
canciones_df = pd.read_csv('canciones.csv')

artistas_lastfm_df.to_sql('artistas_lastfm', conn, if_exists='replace', index=False)
albumes_df.to_sql('albumes_lastfm', conn, if_exists='replace', index=False)
canciones_df.to_sql('canciones_lastfm', conn, if_exists='replace', index=False)
print("Datos de LastFM insertados en la base de datos.")

Datos de LastFM insertados en la base de datos.


In [78]:
import requests

API_KEY = '11e26b6792a6279c70bb7b3771993233'
ARTISTA = "Madonna"

# Buscar información de Madonna en LastFM
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={ARTISTA}&api_key={API_KEY}&format=json"
response = requests.get(url)
data = response.json()

if 'artist' in data:
    artista_lastfm = data['artist']
    artista = {
        'Artist': artista_lastfm.get('name'),
        'Biography': artista_lastfm.get('bio', {}).get('summary'),
        'Listeners': artista_lastfm.get('stats', {}).get('listeners'),
        'Playcount': artista_lastfm.get('stats', {}).get('playcount'),
        'Similar_Artists': ', '.join([a['name'] for a in artista_lastfm.get('similar', {}).get('artist', [])])
    }
else:
    artista = {}

artista

{'Artist': 'Madonna',
 'Biography': 'Madonna Louise Veronica Ciccone (born August 16, 1958 in Bay City, Michigan) is an American singer, songwriter and actress. She is considered one of the most influential figures in popular culture and has often been referred to as the "Queen of Pop". Madonna is noted for her continual reinvention and versatility in music production, songwriting and visual presentation. She is also known for pushing the boundaries of artistic expression in mainstream music, while maintaining control over every aspect of her career. <a href="https://www.last.fm/music/Madonna">Read more on Last.fm</a>',
 'Listeners': '5323677',
 'Playcount': '338504766',
 'Similar_Artists': 'Kylie Minogue, Janet Jackson, Britney Spears, Dannii Minogue, Paula Abdul'}

In [79]:
import requests

API_KEY = '11e26b6792a6279c70bb7b3771993233'
ARTISTA = "Madonna"

# Buscar artistas similares a Madonna en LastFM
url = f"http://ws.audioscrobbler.com/2.0/?method=artist.getsimilar&artist={ARTISTA}&api_key={API_KEY}&format=json"
response = requests.get(url)
data = response.json()

if 'similarartists' in data and 'artist' in data['similarartists']:
    similar_artists = [artist['name'] for artist in data['similarartists']['artist']]
else:
    similar_artists = []
similar_artists

['Kylie Minogue',
 'Janet Jackson',
 'Britney Spears',
 'Dannii Minogue',
 'Paula Abdul',
 'Cher',
 'Christina Aguilera',
 'Sophie Ellis-Bextor',
 'Lady Gaga',
 'Jennifer Lopez',
 'Mariah Carey',
 'Robyn',
 'Spice Girls',
 'Donna Summer',
 'Cyndi Lauper',
 'Gwen Stefani',
 'Whitney Houston',
 'Melanie C',
 'Geri Halliwell',
 'Tina Turner',
 'Céline Dion',
 'Rihanna',
 'Taylor Dayne',
 'Shakira',
 'Katy Perry',
 'Róisín Murphy',
 'Miley Cyrus',
 'Grace Jones',
 'Ke$ha',
 'Girls Aloud',
 'Nelly Furtado',
 'Jessie Ware',
 'Kate Bush',
 'Michael Jackson',
 'Cathy Dennis',
 'Fergie',
 'Alanis Morissette',
 'George Michael',
 'Goldfrapp',
 'Barbra Streisand',
 'Mylène Farmer',
 'Sugababes',
 'Shania Twain',
 'Addison Rae',
 'Heidi Montag',
 'Paris Hilton',
 'Annie Lennox',
 'Bananarama',
 'Emma Bunton',
 'Tove Lo',
 'Lindsay Lohan',
 "Destiny's Child",
 'Diana Ross',
 'The Pussycat Dolls',
 'Kelly Osbourne',
 'Rachel Stevens',
 'Beyoncé',
 'Jessica Simpson',
 'Crystal Waters',
 'Deee-Lite',


In [80]:
artista['similar_artists'] = similar_artists
artista 

{'Artist': 'Madonna',
 'Biography': 'Madonna Louise Veronica Ciccone (born August 16, 1958 in Bay City, Michigan) is an American singer, songwriter and actress. She is considered one of the most influential figures in popular culture and has often been referred to as the "Queen of Pop". Madonna is noted for her continual reinvention and versatility in music production, songwriting and visual presentation. She is also known for pushing the boundaries of artistic expression in mainstream music, while maintaining control over every aspect of her career. <a href="https://www.last.fm/music/Madonna">Read more on Last.fm</a>',
 'Listeners': '5323677',
 'Playcount': '338504766',
 'Similar_Artists': 'Kylie Minogue, Janet Jackson, Britney Spears, Dannii Minogue, Paula Abdul',
 'similar_artists': ['Kylie Minogue',
  'Janet Jackson',
  'Britney Spears',
  'Dannii Minogue',
  'Paula Abdul',
  'Cher',
  'Christina Aguilera',
  'Sophie Ellis-Bextor',
  'Lady Gaga',
  'Jennifer Lopez',
  'Mariah Carey

In [81]:
# Consulta: artistas y oyentes ordenados de mayor a menor
c.execute("""
    SELECT nombre, oyentes
    FROM artistas_lastfm
    ORDER BY oyentes DESC
""")
for row in c.fetchall():
    print(row)

('Coldplay', 8770305)
('Coldplay', 8770305)
('Coldplay', 8768824)
('Coldplay', 8768824)
('Coldplay', 8768824)
('Coldplay', 8768824)
('Coldplay', 8765549)
('Adele', 5392663)
('Adele', 5392663)
('Adele', 5391306)
('Adele', 5391306)
('Adele', 5391306)
('Adele', 5391306)
('Adele', 5389065)
('Shakira', 4600712)
('Shakira', 4600712)
('Shakira', 4599368)
('Shakira', 4599368)
('Shakira', 4599368)
('Shakira', 4599368)
('Shakira', 4596372)
('Bad Bunny', 2019745)
('Bad Bunny', 2019745)
('Bad Bunny', 2017111)
('Bad Bunny', 2017111)
('Bad Bunny', 2017111)
('Bad Bunny', 2017111)
('Bad Bunny', 2011677)
